This notebook will be used mainly for the Coursera Capstone project

# Determining the location for a new gym in Brooklyn

# Introduction

During the Covid-19 pandemic, the gym sector has suffered a big drop in the revenues due to the restrictions put in place by governments to limit infections. 
Even though many fitness enthusiasts created their own home gym to keep on training during the lockdown phase, the gym represents in the heart of many, not only a place where you can train with other people, but also an environment where you can socialise and create meaningful relationships. Those relationships that the pandemic has taught us to be very important. 

# Business Problem

Having said that the gym sector has been hardly beaten by social restrictions in the last months, it's important to make good strategic decisions not to loose even more money. 
The purchase/rent of the building where the gym will be located, is definitely the biggest and the most important a gym owner has to make. Therefore, it is fundamental to take in consideration as many factors has possibile in the choice of the location, as from it will depend a large part of the future of the business. 
PumpIt, is an international company that owns gyms in all the major European capitals; their goal for 2021 is target the US market starting from New York City and then expanding in the West Coast. More precisely, they have picked Brooklyn to be the starting point for their expansion project in the US. 
With a surface of 180 km^2, Brooklyn is the second largest borough in New York City after Queens; scanning such a big area to find the best location for a new gym, is not an easy task. This is why the company has hired as consultants, a team of data scientist to be supported during the decision making process with meaningful insights coming from data. 
What consultants have been asked to do, is not only to understand which is the best area in Brooklyn where to open a new gym, but also to check the gym rating nearby the chosen area to understand a little bit more about the competition.

# Data

The Data that will be used for this analysis are mainly of two types: 

1. Data about New York City neighbourhoods and borough with their longitude and latitude to be able to create maps

2. Venues Data retrieved using the Footsquare API to explore and analyse neighbourhoods in New York City, more specifically those located in the Brooklyn borough

# CODE

The first thing to be done is to import all the libraries that we will need to conduct our analysis

In [47]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
! pip install folium==0.5.0
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


# Data

What we have to do next is to import and then load the data to be used for our analysis; first, data about boroughs and neighborhoods of New York City together with their postal code, latitude and longitude

In [11]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [12]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [13]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

It is possible to notice that the important data (neighborhoods) are in the features key; therefore, we create an object that will contain these type of data

In [14]:
neighborhoods_data = newyork_data['features']

What we have to do next is to turn deta downloaded as a Json file, into a Pandas Dataframe.
First, we have to create the df that will contain them

In [15]:
# we define the columns name
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# and create the df
neighborhoods = pd.DataFrame(columns=column_names)
# we can have a look at the df empty, just with columns
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude


Now we can fill our df with the data we have downloaded, with the help of a for cicle

In [16]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

This is what our first df looks like

In [17]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let's also understand something more about the data.
For example we may get the tot number of Burough and Neighborhoods.

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


# Some initial visualization and data selection

By introdcuing the longitude and the latitude of New York City, we are able to create a visual represenation of the neighborhoods

In [23]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

newyork_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newyork_map)  
    
newyork_map

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


For the sake of this work, we will focus only on the boroguh of Brooklyn as explained in the Business Problem section

In [24]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


Similarly to what we have done for New York, we can import the longitude and the latitude of Brooklyn and visualize its neighborhoods

In [108]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

map_Brooklyn  = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Brooklyn )  
    
map_Brooklyn 

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


As explained in the section "Data", the second type of data we will use deal wih venues in New York City, more specifically in Brooklyn, and are retrived using Footsquare Api. Therefore, we have to set our credetials to make calls

In [34]:
CLIENT_ID = 'JEAKFVM53VF2CZLUYTO2VCL1QVYVSFNWLABBR2VN2XTL2SYL'
CLIENT_SECRET = 'UAVJXVWFHGWIHBUJE13RGHBC3FM0TICWSBW2V0SNHIE4VP2Z' 
VERSION = '20180605' 
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JEAKFVM53VF2CZLUYTO2VCL1QVYVSFNWLABBR2VN2XTL2SYL
CLIENT_SECRET:UAVJXVWFHGWIHBUJE13RGHBC3FM0TICWSBW2V0SNHIE4VP2Z


Now we can explore all the venues in Brooklyn with a limit of 100 venues within a radius of 10km

In [35]:
LIMIT = 100
radius = 10000
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


Let's visualize the resulting df...

In [36]:
print(Brooklyn_venues.shape)
Brooklyn_venues.head()

(2740, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
4,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot


... and check the number of venues per Neighborhood

In [37]:
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,52,52,52,52,52,52
Bay Ridge,84,84,84,84,84,84
Bedford Stuyvesant,34,34,34,34,34,34
Bensonhurst,28,28,28,28,28,28
Bergen Beach,6,6,6,6,6,6
Boerum Hill,89,89,89,89,89,89
Borough Park,24,24,24,24,24,24
Brighton Beach,42,42,42,42,42,42
Broadway Junction,15,15,15,15,15,15


# Neighborhoods analysis

To provide our client with a good indication on where to place its gym, we have to go deep on each neighborhood to understand what kind of activities are performed in every area.
This can be done by creating dummies for types of venues per each neighborhood.

In [40]:
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 

fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Rooftop Bar,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attract

This type of visualization is not so clear, therefore we can compute the mean frequencies of the occurence, grouped by neighborhood...

In [42]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Rooftop Bar,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attract

... and then the 10 most common venues per neighborhood

In [43]:
num_top_venues = 10

for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                   venue  freq
0        Bubble Tea Shop  0.06
1     Chinese Restaurant  0.06
2             Donut Shop  0.04
3   Fast Food Restaurant  0.04
4            Gas Station  0.04
5            Pizza Place  0.04
6               Pharmacy  0.04
7     Italian Restaurant  0.04
8   Cantonese Restaurant  0.02
9  Check Cashing Service  0.02


----Bay Ridge----
                 venue  freq
0                  Spa  0.06
1   Italian Restaurant  0.06
2          Pizza Place  0.05
3     Greek Restaurant  0.04
4           Bagel Shop  0.04
5                  Bar  0.04
6  American Restaurant  0.04
7             Pharmacy  0.02
8                Diner  0.02
9        Grocery Store  0.02


----Bedford Stuyvesant----
                     venue  freq
0                      Bar  0.09
1            Deli / Bodega  0.09
2              Coffee Shop  0.09
3                     Café  0.06
4              Pizza Place  0.06
5                Juice Bar  0.03
6   Thrift / Vintage Store  0.03
7       

                 venue  freq
0                 Park  0.10
1  American Restaurant  0.05
2          Coffee Shop  0.05
3       Ice Cream Shop  0.05
4            Roof Deck  0.05
5        Boat or Ferry  0.03
6           Playground  0.03
7       Scenic Lookout  0.03
8          Pizza Place  0.03
9               Bridge  0.03


----Georgetown----
             venue  freq
0             Bank  0.11
1         Pharmacy  0.07
2       Donut Shop  0.04
3  Supplement Shop  0.04
4     Burger Joint  0.04
5    Shopping Mall  0.04
6   Breakfast Spot  0.04
7   Shipping Store  0.04
8   Clothing Store  0.04
9        Pet Store  0.04


----Gerritsen Beach----
                 venue  freq
0                  Bar  0.16
1          Pizza Place  0.11
2           Bagel Shop  0.05
3  Martial Arts School  0.05
4          Event Space  0.05
5         Skating Rink  0.05
6          Gas Station  0.05
7       Ice Cream Shop  0.05
8        Deli / Bodega  0.05
9     Department Store  0.05


----Gowanus----
                    ve

                 venue  freq
0          Bus Station  0.17
1             Pharmacy  0.08
2          Pizza Place  0.08
3         Intersection  0.08
4  American Restaurant  0.08
5          Supermarket  0.08
6             Bus Stop  0.08
7           Donut Shop  0.08
8        Shopping Mall  0.08
9   Weight Loss Center  0.08


----Sunset Park----
                       venue  freq
0                     Bakery  0.08
1  Latin American Restaurant  0.08
2         Mexican Restaurant  0.08
3                Pizza Place  0.08
4                       Bank  0.08
5          Mobile Phone Shop  0.05
6                   Pharmacy  0.05
7              Deli / Bodega  0.05
8             Ice Cream Shop  0.05
9                        Gym  0.05


----Vinegar Hill----
                      venue  freq
0                Food Truck  0.14
1               Coffee Shop  0.07
2  Bike Rental / Bike Share  0.07
3                      Café  0.07
4            Ice Cream Shop  0.04
5                  Pharmacy  0.04
6     Perform

In [ ]:
 We can sort the venues in descending order and create a df that displays the top 15 venues per neighborhood

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 15

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Bath Beach,Chinese Restaurant,Bubble Tea Shop,Pharmacy,Gas Station,Pizza Place,Fast Food Restaurant,Donut Shop,Italian Restaurant,Burger Joint,Middle Eastern Restaurant,Mobile Phone Shop,Flower Shop,Sandwich Place,Peruvian Restaurant,Liquor Store
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bar,Bagel Shop,Diner,Pharmacy,Café,Sandwich Place,Chinese Restaurant,New American Restaurant,Sushi Restaurant,Playground
2,Bedford Stuyvesant,Coffee Shop,Deli / Bodega,Bar,Café,Pizza Place,Gourmet Shop,Playground,Boutique,Gift Shop,Wine Bar,Cocktail Bar,Basketball Court,Bus Stop,Community Center,Wine Shop
3,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Flower Shop,Donut Shop,Italian Restaurant,Sushi Restaurant,Sporting Goods Shop,Cha Chaan Teng,Supermarket,Coffee Shop,Shabu-Shabu Restaurant,Park,Bakery,Bagel Shop,Pet Store
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Athletics & Sports,Park,Field,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Filipino Restaurant,Fish Market,Flea Market


# Clustering Neighborhoods

The next step is to consolidate different neighborhoods into clusters to split the areas within the Brooklyn borough.
This will help us in understanding where is the best place to open a new gym, as a fucntion of the other venues in the area.
Keeping in mind that in the Lab we have split Manhattan into 5 clusters, I've picked 7 to be k as Brooklyn's area is larger than Manhattan one.

In [48]:
kclusters = 7
Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 5, 5, 1, 6, 5, 1, 5, 1, 5], dtype=int32)

Now we merge the dataset including the top 15 venues per neighborhood with the one containing information about clusters.
This will help us to understand which types of venues are presente in each cluster.

In [49]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_data

Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,5,Spa,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bar,Bagel Shop,Diner,Pharmacy,Café,Sandwich Place,Chinese Restaurant,New American Restaurant,Sushi Restaurant,Playground
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Chinese Restaurant,Ice Cream Shop,Flower Shop,Donut Shop,Italian Restaurant,Sushi Restaurant,Sporting Goods Shop,Cha Chaan Teng,Supermarket,Coffee Shop,Shabu-Shabu Restaurant,Park,Bakery,Bagel Shop,Pet Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Latin American Restaurant,Pizza Place,Bank,Bakery,Mexican Restaurant,Pharmacy,Ice Cream Shop,Deli / Bodega,Gym,Mobile Phone Shop,Fried Chicken Joint,Chinese Restaurant,Supplement Shop,Grocery Store,Breakfast Spot
3,Brooklyn,Greenpoint,40.730201,-73.954241,5,Bar,Coffee Shop,Pizza Place,Cocktail Bar,Yoga Studio,French Restaurant,Furniture / Home Store,Record Shop,Garden Center,Restaurant,Polish Restaurant,Sandwich Place,Deli / Bodega,Mexican Restaurant,Café
4,Brooklyn,Gravesend,40.595260,-73.973471,5,Italian Restaurant,Pizza Place,Lounge,Bar,Bakery,Restaurant,Gym,Pharmacy,Furniture / Home Store,Sporting Goods Shop,Bus Station,Donut Shop,Flea Market,Fish Market,Factory


We can also get a visual map of how the clusters have been split 

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In this section I will present and comment the result of the clustering process and provide the final advice on which is the best area of Brooklyn to open a new gym.

# Cluster 1: Resturants

In [53]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 0, Brooklyn_merged.columns[[1] + list(range(5,Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
28,Canarsie,Deli / Bodega,Caribbean Restaurant,Thai Restaurant,Asian Restaurant,Gym,Yemeni Restaurant,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market


# Cluster 2: Complete Services 

In [54]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 1, Brooklyn_merged.columns[[1] + list(range(5,Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Flower Shop,Donut Shop,Italian Restaurant,Sushi Restaurant,Sporting Goods Shop,Cha Chaan Teng,Supermarket,Coffee Shop,Shabu-Shabu Restaurant,Park,Bakery,Bagel Shop,Pet Store
2,Sunset Park,Latin American Restaurant,Pizza Place,Bank,Bakery,Mexican Restaurant,Pharmacy,Ice Cream Shop,Deli / Bodega,Gym,Mobile Phone Shop,Fried Chicken Joint,Chinese Restaurant,Supplement Shop,Grocery Store,Breakfast Spot
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Cosmetics Shop,Grocery Store,Mobile Phone Shop,Steakhouse,Organic Grocery,Chinese Restaurant,Gourmet Shop,Bank,Bakery,Bagel Shop,Spa,Restaurant
8,Flatbush,Deli / Bodega,Caribbean Restaurant,Coffee Shop,Pharmacy,Middle Eastern Restaurant,Mexican Restaurant,Lounge,Chinese Restaurant,Bank,Bagel Shop,Juice Bar,Sandwich Place,Donut Shop,Pizza Place,Plaza
10,East Flatbush,Chinese Restaurant,Moving Target,Liquor Store,Pharmacy,Park,Caribbean Restaurant,Department Store,Supermarket,Print Shop,Health Food Store,Farm,Ethiopian Restaurant,Event Service,Event Space,Factory
14,Brownsville,Fried Chicken Joint,Burger Joint,Convenience Store,Farmers Market,Chinese Restaurant,Spanish Restaurant,Park,Trail,Performing Arts Venue,Restaurant,Pharmacy,Pool,Pizza Place,Playground,Plaza
25,Cypress Hills,Fried Chicken Joint,Metro Station,Ice Cream Shop,Fast Food Restaurant,Pizza Place,Gas Station,Baseball Field,Supermarket,Bank,South American Restaurant,Mobile Phone Shop,Donut Shop,Mexican Restaurant,Latin American Restaurant,Spanish Restaurant
26,East New York,Deli / Bodega,Plaza,Caribbean Restaurant,Gym,Fast Food Restaurant,Pizza Place,Bus Station,Spanish Restaurant,Fried Chicken Joint,Event Service,Flea Market,Fish Market,Factory,Fish & Chips Shop,Filipino Restaurant
27,Starrett City,Bus Station,Bus Stop,Caribbean Restaurant,American Restaurant,Intersection,Weight Loss Center,Supermarket,Pizza Place,Shopping Mall,Donut Shop,Pharmacy,Filipino Restaurant,Farmers Market,Fast Food Restaurant,Field
29,Flatlands,Pharmacy,Caribbean Restaurant,Fast Food Restaurant,Fried Chicken Joint,Paper / Office Supplies Store,Video Store,Bar,Park,Lounge,Check Cashing Service,Nightclub,Intersection,Dessert Shop,Discount Store,Fish Market


# Cluster 3: Parks and resturants

In [55]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 2, Brooklyn_merged.columns[[1] + list(range(5,Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
21,Red Hook,Seafood Restaurant,Park,Art Gallery,Pub,Bagel Shop,Farm,Café,Flower Shop,Brewery,Bar,Pizza Place,American Restaurant,Wine Shop,Record Shop,Antique Shop
31,Manhattan Beach,Café,Harbor / Marina,Ice Cream Shop,Sandwich Place,Bus Stop,Playground,Pizza Place,Beach,Yemeni Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
35,Dyker Heights,Burger Joint,Golf Course,Cosmetics Shop,Bagel Shop,Park,Yemeni Restaurant,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Filipino Restaurant,Fish Market,Flea Market
39,Sea Gate,Sports Club,Spa,Bus Station,Bus Line,Home Service,Beach,American Restaurant,Food & Drink Shop,Food Court,Food Stand,Falafel Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
61,Fulton Ferry,Park,American Restaurant,Roof Deck,Coffee Shop,Ice Cream Shop,Bridge,Bar,Scenic Lookout,Bakery,Pizza Place,Playground,Boat or Ferry,Board Shop,Food & Drink Shop,Food Court
62,Vinegar Hill,Food Truck,Bike Rental / Bike Share,Café,Coffee Shop,Factory,Scenic Lookout,Latin American Restaurant,Bookstore,Bar,Bakery,Park,Sandwich Place,Entertainment Service,Performing Arts Venue,Whisky Bar
65,Dumbo,Park,Bakery,Coffee Shop,American Restaurant,Scenic Lookout,Ice Cream Shop,Pizza Place,Roof Deck,Café,Boxing Gym,Bookstore,Italian Restaurant,Bar,Yoga Studio,Gym


# Cluster 4: Complete services

In [56]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3, Brooklyn_merged.columns[[1] + list(range(5,Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
30,Mill Island,Pool,Locksmith,Yemeni Restaurant,Filipino Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Fish & Chips Shop,Flea Market,Flower Shop,Food


# Cluster 5: Business

In [57]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4, Brooklyn_merged.columns[[1] + list(range(5,Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
59,Paerdegat Basin,Business Service,Food,Gym,Child Care Service,Food & Drink Shop,Food Court,Flower Shop,Flea Market,Fish Market,Food Stand,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market


# Cluster 6: Complete Services

In [58]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 5, Brooklyn_merged.columns[[1] + list(range(5,Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bar,Bagel Shop,Diner,Pharmacy,Café,Sandwich Place,Chinese Restaurant,New American Restaurant,Sushi Restaurant,Playground
3,Greenpoint,Bar,Coffee Shop,Pizza Place,Cocktail Bar,Yoga Studio,French Restaurant,Furniture / Home Store,Record Shop,Garden Center,Restaurant,Polish Restaurant,Sandwich Place,Deli / Bodega,Mexican Restaurant,Café
4,Gravesend,Italian Restaurant,Pizza Place,Lounge,Bar,Bakery,Restaurant,Gym,Pharmacy,Furniture / Home Store,Sporting Goods Shop,Bus Station,Donut Shop,Flea Market,Fish Market,Factory
5,Brighton Beach,Russian Restaurant,Beach,Gourmet Shop,Bank,Restaurant,Sushi Restaurant,Donut Shop,Pharmacy,Mobile Phone Shop,Eastern European Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Non-Profit,Bookstore,Supermarket
6,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Yoga Studio,Diner,Café,Sandwich Place,Russian Restaurant,Restaurant,Creperie,Pizza Place,Beer Garden,Outlet Store,Miscellaneous Shop,Grocery Store,Karaoke Bar
9,Crown Heights,Pizza Place,Museum,Café,Bakery,Salon / Barbershop,Coffee Shop,Supermarket,Candy Store,Kosher Restaurant,Bagel Shop,Burger Joint,Cosmetics Shop,Deli / Bodega,Playground,Dry Cleaner
11,Kensington,Thai Restaurant,Grocery Store,Pizza Place,Ice Cream Shop,Massage Studio,Coffee Shop,Outdoors & Recreation,Liquor Store,Furniture / Home Store,Taco Place,Supermarket,Park,Sandwich Place,Mexican Restaurant,Spa
12,Windsor Terrace,Plaza,Diner,Deli / Bodega,Park,Grocery Store,Bagel Shop,Chinese Restaurant,Café,Butcher,Food Truck,Bookstore,Sushi Restaurant,Beer Store,Thrift / Vintage Store,Bar
13,Prospect Heights,Bar,Mexican Restaurant,Thai Restaurant,Cocktail Bar,Coffee Shop,Bakery,Restaurant,Wine Shop,Diner,Gourmet Shop,Café,Southern / Soul Food Restaurant,New American Restaurant,Ice Cream Shop,Wine Bar
15,Williamsburg,Pizza Place,Bagel Shop,Burger Joint,Tapas Restaurant,Liquor Store,Taco Place,Lounge,Steakhouse,Breakfast Spot,Middle Eastern Restaurant,Café,Greek Restaurant,Sandwich Place,Event Space,Clothing Store


# Cluster 7: Free time

In [59]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 6, Brooklyn_merged.columns[[1] + list(range(5,Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
45,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Athletics & Sports,Park,Field,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Filipino Restaurant,Fish Market,Flea Market


# Clustering Results

Clustering results show that Cluster 2, Cluster 4 and Cluster 6 include a cast variety of venues and services:this may mean that a gym opening would attract many indivudals with very different life-styles and interets. However, it may result in an anonimus choice with little risonance due to the large set of venues available.
Cluster 1 seems to be mainly dominated by resturants of all types; this indicates that is not the best environmet to place a gym as people coming in this area are intrested by other types of venues.
Cluster 3 includes mainly Parks and resturants. It sounds attractive but at the same time we have to keep in mind that if there is the possiblity to train outdoor for free, people may avoid paying a gym subscription when they can perform their workout at the fresh air without spending their money.
Cluster 5 is mainly focused on business, as it displays busienss services, childe care services and food in general. It may makes sense to open a gym available 24h for workers nearby the place they work at. This would save them time to get to the gym therefore increasing the motivation.
Finally, Cluster 7 is mainly characterized by the presence of venues for free time and sports. For instance, we can find: basketball field, playground, athletics & sports and parks. At the same time, we can find factories that includes the advantage of workers presence.

# Final Advice

My final advice would be to pick Cluster 7, more precisely Bergen Beach, as the location for a new gym opening due to the big presence of free time/sport-related venues; the presence of factories may also attract those workers willing to train after/before work.
It may be the case that the competition is strong, but we have to keep in mind that also the influx of clients may be higher.

# What about the competition?

Let's check whether there are any gyms nearby the area that we have identified and if ratings are available.
To do this, we first have to define the latitude and the longitude of Bergen Beach.

In [60]:
address = 'Bergen Beach, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.6203822 -73.9068033


Now we can look for the other gym in the Bergen Beach area that mey be our client's competitor

In [61]:
search_query = 'Gym'
radius = 1000
print(search_query + ' .... OK!')

Gym .... OK!


What we have to do next is to repeat the process of data downloading and preprocessing that we have done before.

In [71]:
ACCESS_TOKEN = 'EDUADLC2ITQCZJZJL2XVJV1B5EOE3QOLLNPK12DX5D3U02HB'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60f57f04f1fb0416ec880b77'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ef2135f30f8e7873b6d8ca2',
    'name': 'Mill Harbor Gym',
    'location': {'address': 'Mill Harbor Condo',
     'crossStreet': 'Ave T',
     'lat': 40.624344,
     'lng': -73.90747,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.624344,
       'lng': -73.90747}],
     'distance': 444,
     'postalCode': '11234',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Mill Harbor Condo (Ave T)',
      'Brooklyn, NY 11234',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1626701572

In [72]:
venues = results['response']['venues']

In [73]:
dataframe = json_normalize(venues)
dataframe.head()
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Mill Harbor Gym,Gym,Mill Harbor Condo,Ave T,40.624344,-73.907470,"[{'label': 'display', 'lat': 40.624344, 'lng':...",444,11234,US,Brooklyn,NY,United States,"[Mill Harbor Condo (Ave T), Brooklyn, NY 11234...",4ef2135f30f8e7873b6d8ca2
1,69th Street Gym,Gym,2350 E 69th St,NaN,40.614644,-73.904911,"[{'label': 'display', 'lat': 40.614644, 'lng':...",658,11234,US,Brooklyn,NY,United States,"[2350 E 69th St, Brooklyn, NY 11234, United St...",4e04c332d22d4cebf3743daf
2,Gott Gym,Gym / Fitness Center,NaN,NaN,40.618824,-73.914084,"[{'label': 'display', 'lat': 40.61882357193908...",639,11234,US,NaN,New York,United States,"[NY 11234, United States]",4ed374a2f790d0703c04dca4
3,St Bernard Gymnasium,Gym,69th St,NaN,40.619572,-73.909691,"[{'label': 'display', 'lat': 40.61957170000001...",260,11234,US,Brooklyn,NY,United States,"[69th St, Brooklyn, NY 11234, United States]",4c5766d1cc96c9b693ff782e
4,Harbor Fitness,Gym,6161 Strickland Ave,at Mill Avenue,40.612574,-73.912844,"[{'label': 'display', 'lat': 40.61257356306318...",1008,11234,US,Brooklyn,NY,United States,"[6161 Strickland Ave (at Mill Avenue), Brookly...",4b5a54c4f964a520cebd28e3


The following, are the 5 gyms in cluster 7 area 

In [74]:
dataframe_filtered.name

0         Mill Harbor Gym
1         69th Street Gym
2                Gott Gym
3    St Bernard Gymnasium
4          Harbor Fitness
Name: name, dtype: object

Let's see them on the map

In [75]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

Let's try to understand a little bit more about these gyms by finding their rating

# Gym 1: Mill Harbor Gym

In [89]:
venue_id_1 = '4ef2135f30f8e7873b6d8ca2'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_1, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ef2135f30f8e7873b6d8ca2',
 'name': 'Mill Harbor Gym',
 'contact': {},
 'location': {'address': 'Mill Harbor Condo',
  'crossStreet': 'Ave T',
  'lat': 40.624344,
  'lng': -73.90747,
  'labeledLatLngs': [{'label': 'display', 'lat': 40.624344, 'lng': -73.90747}],
  'postalCode': '11234',
  'cc': 'US',
  'city': 'Brooklyn',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['Mill Harbor Condo (Ave T)',
   'Brooklyn, NY 11234',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/mill-harbor-gym/4ef2135f30f8e7873b6d8ca2',
 'categories': [{'id': '4bf58dd8d48988d176941735',
   'name': 'Gym',
   'pluralName': 'Gyms',
   'shortName': 'Gym',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
    'count': 3,
    'items': [{'id': '387719078',
      'firstName': 'F',
      'lastName': 'M

In [90]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


As a rating is still not available, we can look at the number of likes

In [94]:
print(result['response']['venue']['likes']['count'])

3


# Gym 2: 69th Street Gym

In [96]:
venue_id_1 = '4e04c332d22d4cebf3743daf'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_1, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4e04c332d22d4cebf3743daf',
 'name': '69th Street Gym',
 'contact': {'phone': '+17184447455', 'formattedPhone': '+1 718-444-7455'},
 'location': {'address': '2350 E 69th St',
  'lat': 40.614644,
  'lng': -73.904911,
  'labeledLatLngs': [{'label': 'display', 'lat': 40.614644, 'lng': -73.904911},
   {'label': 'entrance', 'lat': 40.614768, 'lng': -73.904722}],
  'postalCode': '11234',
  'cc': 'US',
  'city': 'Brooklyn',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['2350 E 69th St',
   'Brooklyn, NY 11234',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/69th-street-gym/4e04c332d22d4cebf3743daf',
 'categories': [{'id': '4bf58dd8d48988d176941735',
   'name': 'Gym',
   'pluralName': 'Gyms',
   'shortName': 'Gym',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'likes': {'count': 1,
  'groups': [{'type': 'others',
    'coun

In [97]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


As previously, let's look at the number of likes

In [98]:
print(result['response']['venue']['likes']['count'])

1


# Gym 3: Gott Gym

In [99]:
venue_id_1 = '4ed374a2f790d0703c04dca4'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_1, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ed374a2f790d0703c04dca4',
 'name': 'Gott Gym',
 'contact': {},
 'location': {'lat': 40.61882357193908,
  'lng': -73.91408357193033,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.61882357193908,
    'lng': -73.91408357193033}],
  'postalCode': '11234',
  'cc': 'US',
  'state': 'New York',
  'country': 'United States',
  'formattedAddress': ['NY 11234', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/gott-gym/4ed374a2f790d0703c04dca4',
 'categories': [{'id': '4bf58dd8d48988d175941735',
   'name': 'Gym / Fitness Center',
   'pluralName': 'Gyms or Fitness Centers',
   'shortName': 'Gym / Fitness',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'likes': {'count': 0, 'groups': []},
 'like': False,
 'dislike': False,
 'ok': False,
 'allowMenuUrlEdit': True,
 'beenHere': {'count': 0,
  'unconfirmedCount': 0,
  'marked': False,
  'lastCheck

In [100]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [101]:
print(result['response']['venue']['likes']['count'])

0


# Gym 4: St Bernard Gymnasium 

In [102]:
venue_id_1 = '4c5766d1cc96c9b693ff782e' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_1, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4c5766d1cc96c9b693ff782e',
 'name': 'St Bernard Gymnasium',
 'contact': {},
 'location': {'address': '69th St',
  'lat': 40.61957170000001,
  'lng': -73.90969086666666,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.61957170000001,
    'lng': -73.90969086666666}],
  'postalCode': '11234',
  'cc': 'US',
  'city': 'Brooklyn',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['69th St', 'Brooklyn, NY 11234', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/st-bernard-gymnasium/4c5766d1cc96c9b693ff782e',
 'categories': [{'id': '4bf58dd8d48988d176941735',
   'name': 'Gym',
   'pluralName': 'Gyms',
   'shortName': 'Gym',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'likes': {'count': 0, 'groups': []},
 'like': False,
 'dislike': False,
 'ok': False,
 'allowMenuUrlEdit': True,
 'beenHere': {'count': 0,
  'unconfirmedCou

In [103]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [104]:
print(result['response']['venue']['likes']['count'])

0


# Gyme 5: Harbor Fitness

In [105]:
venue_id_1 = '4b5a54c4f964a520cebd28e3' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_1, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4b5a54c4f964a520cebd28e3',
 'name': 'Harbor Fitness',
 'contact': {'phone': '+17182412623',
  'formattedPhone': '+1 718-241-2623',
  'twitter': 'harborfitness'},
 'location': {'address': '6161 Strickland Ave',
  'crossStreet': 'at Mill Avenue',
  'lat': 40.61257356306318,
  'lng': -73.91284403216936,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.61257356306318,
    'lng': -73.91284403216936},
   {'label': 'entrance', 'lat': 40.612657, 'lng': -73.912609}],
  'postalCode': '11234',
  'cc': 'US',
  'city': 'Brooklyn',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['6161 Strickland Ave (at Mill Avenue)',
   'Brooklyn, NY 11234',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harbor-fitness/4b5a54c4f964a520cebd28e3',
 'categories': [{'id': '4bf58dd8d48988d176941735',
   'name': 'Gym',
   'pluralName': 'Gyms',
   'shortName': 'Gym',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
 

In [106]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


Unfourtunately, only 1 of the 5 gyms present in the area of interest has been graded. The grade is just over the sufficiency which means that Harbor Fitness' clients are not so much satisfied. This may indicate low retention rate and the presence of clients willing to change if new players enter the market

In [107]:
print(result['response']['venue']['likes']['count'])

27


It's intresting to note the gym with, by far,the highest number of likes has a rating of only 6.9. We may infer that the competition is not so fierce as we expected and therefroe there is room for growth for new comers.